# 📊 Journal Entry ID Creator

**Automatically create balanced journal entries from your Excel data in 3 simple steps!**

This tool will:
- ✅ Create balanced journal entries (debits = credits)
- ✅ Group lines by date and matching fields
- ✅ Assign Journal IDs to every line
- ✅ Handle minimum 2-line requirement
- ✅ Download results instantly

## 🚀 Instructions:
1. **Run the setup** (click ▶️ on the cell below)
2. **Upload your Excel file** when prompted
3. **Download your results** with Journal IDs added

---


In [ ]:
# @title 🔧 **STEP 1: Setup & Configuration** { display-mode: "form" }
# @markdown Click the ▶️ button to install required packages and load the journal entry creator.

print("🔧 Setting up Journal Entry ID Creator...")
print("=" * 50)

# Install required packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

try:
    install_package("pandas")
    install_package("openpyxl")
    print("✅ Required packages installed successfully!")
except Exception as e:
    print(f"❌ Error installing packages: {e}")

# Import libraries
try:
    import pandas as pd
    import numpy as np
    from itertools import combinations
    from decimal import Decimal, ROUND_HALF_UP
    import io
    from pathlib import Path
    from google.colab import files
    import warnings
    warnings.filterwarnings('ignore')
    print("✅ Libraries imported successfully!")
except Exception as e:
    print(f"❌ Error importing libraries: {e}")

# Load the Journal Entry Creator
class JournalEntryCreator:
    def __init__(self):
        self.journal_lines = None
        self.grouped_entries = {}
        self.unassigned_lines = []
        self.additional_output_lines = []  # rows to append to output (e.g., plug lines)
        self._id_counts = {}
        self.workbook = None
        self.ws_jel = None
        self.ws_ctb = None
        self.ws_type_row = None
        self.ws_header_row = None
        self.ws_data_start_row = None
        self.template_type = 'old'
        self.skip_id_creation = False
        self._plug_lines_added = False

    def _to_decimal(self, value):
        try:
            return Decimal(str(value))
        except Exception:
            return Decimal('0')

    def _sum_decimal(self, series):
        total = Decimal('0')
        for v in series.fillna(0).tolist():
            total += self._to_decimal(v)
        return total.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

    def _sanitize_field_name(self, name):
        s = str(name).strip()
        return ''.join(ch if ch.isalnum() or ch in (' ', '_', '-') else '_' for ch in s).replace(' ', '_')

    def _format_value_token(self, value):
        if pd.isna(value):
            return 'NA'
        if isinstance(value, (pd.Timestamp, np.datetime64)):
            try:
                dt = pd.to_datetime(value)
                return dt.strftime('%Y%m%d')
            except Exception:
                return self._sanitize_field_name(str(value))
        if isinstance(value, (float, int, np.floating, np.integer)):
            return str(value).replace('.', '_')
        return self._sanitize_field_name(str(value))

    def _normalize_header_token(self, token):
        if token is None:
            return ''
        text = str(token)
        text = text.replace('\u00A0', ' ')
        text = ' '.join(text.strip().split())
        return text

    def _canonical(self, token):
        return self._normalize_header_token(token).lower()

    def _find_header_row(self, df_all, required_names, max_scan_rows=10):
        """Find a row in df_all (header=None) that contains all required column names (case/space-insensitive)."""
        required = {self._canonical(n) for n in required_names}
        rows_to_scan = min(max_scan_rows, len(df_all))
        for r in range(rows_to_scan):
            row_vals = [self._normalize_header_token(v) for v in df_all.iloc[r].tolist()]
            canon = {self._canonical(v) for v in row_vals}
            if required.issubset(canon):
                # Build unique headers preserving original tokens (normalized for cleanliness)
                seen = {}
                headers = []
                for v in row_vals:
                    name = v if v != '' else 'Unnamed'
                    count = seen.get(name, 0)
                    unique = f"{name}_{count}" if count > 0 else name
                    seen[name] = count + 1
                    headers.append(unique)
                return r, headers
        return None, None

    def generate_journal_id(self, grouping_fields, group_key, max_len=100):
        fields = list(grouping_fields) if isinstance(grouping_fields, (list, tuple)) else [grouping_fields]
        if isinstance(group_key, tuple):
            values = list(group_key)
        else:
            values = [group_key]
        min_len = min(len(fields), len(values))
        tokens = [self._format_value_token(values[i]) for i in range(min_len)]
        base = 'ID' if not tokens else '-'.join(tokens)
        if len(base) > max_len:
            base = base[:max_len]
        count = self._id_counts.get(base, 0)
        self._id_counts[base] = count + 1
        if count == 0:
            return base
        suffix = f"__{count+1}"
        if len(base) + len(suffix) > max_len:
            base = base[: max_len - len(suffix)]
        return f"{base}{suffix}"
        
    def _normalize_and_deduplicate_columns(self, columns):
        normalized = []
        for col in columns:
            name = '' if pd.isna(col) else str(col).strip()
            if name.lower() in ('', 'nan', 'none'):
                name = 'Unnamed'
            normalized.append(name)
        seen = {}
        unique_cols = []
        for name in normalized:
            count = seen.get(name, 0)
            unique_name = f"{name}_{count}" if count > 0 else name
            seen[name] = count + 1
            unique_cols.append(unique_name)
        return unique_cols
        
    def load_data_from_uploaded_file(self, uploaded_file_data, filename):
        """Load journal lines from uploaded Excel file data"""
        try:
            from openpyxl import load_workbook
            # Try to load as new template first (Journal Entries & Lines sheet)
            try:
                wb = load_workbook(io.BytesIO(uploaded_file_data))
                if 'Journal Entries & Lines' in wb.sheetnames:
                    self.workbook = wb
                    self.ws_jel = wb['Journal Entries & Lines']
                    self.ws_ctb = wb['Comparative Trial Balances'] if 'Comparative Trial Balances' in wb.sheetnames else None
                    self.template_type = 'new'
                    # Read raw sheet to detect header row robustly
                    df_all = pd.read_excel(io.BytesIO(uploaded_file_data), sheet_name='Journal Entries & Lines', header=None)
                    if len(df_all) == 0:
                        print("❌ No data found in Journal Entries & Lines sheet.")
                        return False
                    req = ['Posted Date', 'Account ID', 'Debit Amount', 'Credit Amount']
                    header_row, headers = self._find_header_row(df_all, req, max_scan_rows=10)
                    if header_row is None:
                        print("❌ Required column 'Posted Date' not found in Journal Entries & Lines sheet")
                        return False
                    # header_row is 0-based index where actual field names are found
                    # In new template: row 0 = Required/Optional, row 1 = field names, row 2+ = data
                    # But _find_header_row returns the row with field names (row 1, zero-indexed)
                    self.ws_type_row = header_row  # 0-based row with Required/Optional (row 1 in Excel)
                    self.ws_header_row = header_row + 1  # Field names row in Excel (1-based)
                    self.ws_data_start_row = header_row + 2  # Data starts here in Excel (1-based)
                    df = df_all.iloc[header_row+1:].copy()
                    df.columns = headers[:len(df.columns)]
                    # Normalize critical columns by canonical name lookup
                    # Build mapping from canonical -> actual name
                    canon_map = {self._canonical(col): col for col in df.columns}
                    try:
                        posted_col = canon_map[self._canonical('Posted Date')]
                        acct_col = canon_map[self._canonical('Account ID')]
                        debit_col = canon_map[self._canonical('Debit Amount')]
                        credit_col = canon_map[self._canonical('Credit Amount')]
                    except KeyError:
                        print("❌ Required columns not found after header normalization")
                        return False
                    # Rename to standard names for internal processing
                    df = df.rename(columns={posted_col: 'Posted Date', acct_col: 'Account ID', debit_col: 'Debit Amount', credit_col: 'Credit Amount'})
                    df = df.dropna(how='all')
                    df['Posted Date'] = pd.to_datetime(df['Posted Date'], errors='coerce').dt.normalize()
                    df['Debit Amount'] = pd.to_numeric(df['Debit Amount'], errors='coerce').fillna(0)
                    df['Credit Amount'] = pd.to_numeric(df['Credit Amount'], errors='coerce').fillna(0)
                    df = df.reset_index(drop=True)
                    df['_row_index'] = range(len(df))
                    self.journal_lines = df
                    print(f"✅ Loaded {len(df)} journal lines from 'Journal Entries & Lines' in {filename}")
                    return True
            except Exception:
                # Fall back to old template if anything fails
                pass

            # Old template handling
            df_all = pd.read_excel(io.BytesIO(uploaded_file_data), header=None)
            
            if len(df_all) == 0:
                print("❌ No data found in Excel file.")
                return False
            
            first_row = df_all.iloc[0].values
            second_row = df_all.iloc[1].values if len(df_all) > 1 else None
            
            if second_row is not None and isinstance(second_row[0], str) and 'Posted Date' in str(second_row[0]):
                print("✅ Detected field names in second row")
                template_row = second_row
                df = df_all.iloc[2:].copy()
            elif isinstance(first_row[0], str) and 'Posted Date' in str(first_row[0]):
                print("✅ Detected template headers in first row")
                template_row = first_row
                df = df_all.iloc[1:].copy()
            else:
                print("✅ Using default column structure")
                template_row = ['Posted Date', 'Account ID', 'Debit Amount', 'Credit Amount'] + [f'Optional_{i}' for i in range(len(df_all.columns) - 4)]
                df = df_all.copy()
            
            if len(df) == 0:
                print("❌ No data found. Please add journal line data.")
                return False
            
            df.columns = self._normalize_and_deduplicate_columns(template_row[:len(df.columns)])
            df = df.dropna(how='all')
            
            required_cols = ['Posted Date', 'Account ID', 'Debit Amount', 'Credit Amount']
            for col in required_cols:
                if col not in df.columns:
                    print(f"❌ Required column '{col}' not found")
                    return False
            
            df['Posted Date'] = pd.to_datetime(df['Posted Date'], errors='coerce')
            df['Posted Date'] = df['Posted Date'].dt.normalize()
            df['Debit Amount'] = pd.to_numeric(df['Debit Amount'], errors='coerce').fillna(0)
            df['Credit Amount'] = pd.to_numeric(df['Credit Amount'], errors='coerce').fillna(0)
            df['_row_index'] = range(len(df))
            
            self.journal_lines = df
            print(f"📊 Loaded {len(df)} journal lines from {filename}")
            return True
            
        except Exception as e:
            print(f"❌ Error loading Excel file: {e}")
            return False
    
    def get_optional_fields(self):
        if self.journal_lines is None:
            return []
        
        required_cols = ['Posted Date', 'Account ID', 'Debit Amount', 'Credit Amount', '_row_index']
        optional_cols = []
        
        for col in self.journal_lines.columns:
            if col not in required_cols:
                try:
                    non_empty = self.journal_lines[col].notna() & (self.journal_lines[col] != '') & (self.journal_lines[col] != '[INSERT FIELD NAME]')
                    if non_empty.any():
                        optional_cols.append(col)
                except:
                    continue
        return optional_cols
    
    def validate_balances(self, epsilon=0.01, return_details=False):
        """Validate balances overall, by posted date, and by month.

        If return_details is True, returns a dict with diagnostics:
            {
              'ok': bool,
              'overall': {'debits': float, 'credits': float, 'net': float},
              'by_date': DataFrame with debits, credits, net, diff,
              'unbalanced_dates': DataFrame subset,
              'by_month': DataFrame with debits, credits, net, diff,
              'unbalanced_months': DataFrame subset,
              'messages': [str, ...]
            }
        Otherwise, raises ValueError on failure, returns True on success.
        """
        if self.journal_lines is None or len(self.journal_lines) == 0:
            if return_details:
                return {'ok': False, 'messages': ["No journal lines loaded to validate."]}
            raise ValueError("No journal lines loaded to validate.")
        if 'Posted Date' not in self.journal_lines.columns:
            if return_details:
                return {'ok': False, 'messages': ["Required column 'Posted Date' is missing."]}
            raise ValueError("Required column 'Posted Date' is missing.")

        df = self.journal_lines.copy()
        details = {'messages': []}

        # Overall
        total_debits = self._sum_decimal(df['Debit Amount'])
        total_credits = self._sum_decimal(df['Credit Amount'])
        overall_net = (total_debits - total_credits)
        details['overall'] = {'debits': total_debits, 'credits': total_credits, 'net': overall_net}
        if abs(overall_net) >= epsilon:
            details['messages'].append(
                f"Overall debits and credits do not balance. Total Debits: ${total_debits:,.2f}, Total Credits: ${total_credits:,.2f}, Difference: ${overall_net:,.2f}."
            )

        # Per date
        df['__date__'] = df['Posted Date'].dt.date
        by_date = df.groupby('__date__', dropna=False)[['Debit Amount', 'Credit Amount']].sum()
        by_date = by_date.rename(columns={'Debit Amount': 'debits', 'Credit Amount': 'credits'})
        by_date['debits'] = by_date['debits'].apply(lambda x: Decimal(str(x))).apply(lambda x: x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
        by_date['credits'] = by_date['credits'].apply(lambda x: Decimal(str(x))).apply(lambda x: x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
        by_date['net'] = by_date['debits'] - by_date['credits']
        by_date['diff'] = by_date['net'].abs()
        unbalanced_dates = by_date[by_date['diff'] >= epsilon]
        details['by_date'] = by_date
        details['unbalanced_dates'] = unbalanced_dates
        if len(unbalanced_dates) > 0:
            msg_lines = ["Unbalanced posted dates detected (debits != credits):"]
            for d, row in unbalanced_dates.iterrows():
                msg_lines.append(
                    f"  - {d}: Debits=${row['debits']:,.2f}, Credits=${row['credits']:,.2f}, Difference=${row['net']:,.2f}"
                )
            details['messages'].append("\n".join(msg_lines))

        # Per month
        df['__month__'] = df['Posted Date'].dt.to_period('M')
        by_month = df.groupby('__month__', dropna=False)[['Debit Amount', 'Credit Amount']].sum()
        by_month = by_month.rename(columns={'Debit Amount': 'debits', 'Credit Amount': 'credits'})
        by_month['debits'] = by_month['debits'].apply(lambda x: Decimal(str(x))).apply(lambda x: x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
        by_month['credits'] = by_month['credits'].apply(lambda x: Decimal(str(x))).apply(lambda x: x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
        by_month['net'] = by_month['debits'] - by_month['credits']
        by_month['diff'] = by_month['net'].abs()
        unbalanced_months = by_month[by_month['diff'] >= epsilon]
        details['by_month'] = by_month
        details['unbalanced_months'] = unbalanced_months
        if len(unbalanced_months) > 0:
            msg_lines = ["Unbalanced posted months detected (debits != credits):"]
            for m, row in unbalanced_months.iterrows():
                msg_lines.append(
                    f"  - {m}: Debits=${row['debits']:,.2f}, Credits=${row['credits']:,.2f}, Difference=${row['net']:,.2f}"
                )
            details['messages'].append("\n".join(msg_lines))

        # Cleanup
        df.drop(columns=['__date__', '__month__'], inplace=True)

        ok = len(details['messages']) == 0
        details['ok'] = ok
        if return_details:
            return details
        if not ok:
            raise ValueError("\n\n".join(details['messages']))
        return True
    
    def check_balance(self, group_df):
        if len(group_df) < 2:
            return False
        total_debits = self._sum_decimal(group_df['Debit Amount'])
        total_credits = self._sum_decimal(group_df['Credit Amount'])
        return total_debits == total_credits
    
    def generate_grouping_combinations(self, optional_fields, max_fields=5):
        combinations_to_try = []
        for r in range(min(len(optional_fields), max_fields), 0, -1):
            for combo in combinations(optional_fields, r):
                combinations_to_try.append(['Posted Date'] + list(combo))
        combinations_to_try.append(['Posted Date'])
        return combinations_to_try
    
    def balance_unassigned_with_plug(self, plug_account_id="Audit Sight Clearing", epsilon=0.01):
        """Create balancing journal entries per posted date for unassigned lines by adding a plug line.

        - Groups unassigned lines by Posted Date (date-only)
        - For each date group, creates a new JE containing those lines
        - Adds one plug line with Account ID = plug_account_id for the offset amount
        - Returns number of dates balanced
        """
        if self.journal_lines is None:
            print("No data loaded. Cannot balance.")
            return 0
        if len(self.unassigned_lines) == 0:
            print("No unassigned lines to balance.")
            return 0

        dates_balanced = 0
        next_id_num = 1
        if len(self.grouped_entries) > 0:
            try:
                nums = [int(k[2:]) for k in self.grouped_entries.keys() if str(k).startswith("JE")]
                if nums:
                    next_id_num = max(nums) + 1
            except Exception:
                pass

        df_un = self.unassigned_lines.copy()
        df_un['__date__'] = df_un['Posted Date'].dt.date
        for date_value, group in df_un.groupby('__date__'):
            group_clean = group.copy().reset_index(drop=True)
            total_debits = self._sum_decimal(group_clean['Debit Amount'])
            total_credits = self._sum_decimal(group_clean['Credit Amount'])
            net = total_debits - total_credits

            je_id = f"JE{next_id_num:04d}"
            next_id_num += 1

            plug_debit = Decimal('0.00')
            plug_credit = Decimal('0.00')
            if net != Decimal('0.00'):
                if net > 0:
                    plug_credit = net.copy_abs().quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
                else:
                    plug_debit = net.copy_abs().quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

                plug_row = {col: None for col in self.journal_lines.columns}
                plug_row['Posted Date'] = pd.to_datetime(date_value)
                plug_row['Account ID'] = plug_account_id
                plug_row['Debit Amount'] = float(plug_debit)
                plug_row['Credit Amount'] = float(plug_credit)
                plug_row['_row_index'] = -1

                group_with_plug = pd.concat([group_clean, pd.DataFrame([plug_row])], ignore_index=True)
            else:
                group_with_plug = group_clean

            self.grouped_entries[je_id] = {
                'lines': group_with_plug.reset_index(drop=True),
                'grouping_fields': ['Posted Date'],
                'group_key': str(date_value),
                'total_debits': float(group_with_plug['Debit Amount'].sum()),
                'total_credits': float(group_with_plug['Credit Amount'].sum())
            }

            # Record additional output line for plug
            if net != Decimal('0.00'):
                add_line = {'Journal ID': je_id}
                add_line['Posted Date'] = pd.to_datetime(date_value)
                add_line['Account ID'] = plug_account_id
                add_line['Debit Amount'] = float(plug_debit)
                add_line['Credit Amount'] = float(plug_credit)
                # Include any extra columns present in data with None defaults
                for col in self.journal_lines.columns:
                    if col in ('Posted Date', 'Account ID', 'Debit Amount', 'Credit Amount', '_row_index'):
                        continue
                    add_line[col] = None
                self.additional_output_lines.append(add_line)
                self._plug_lines_added = True

            dates_balanced += 1

        self.unassigned_lines = self.journal_lines.iloc[0:0].copy()
        return dates_balanced

    def add_plug_lines_for_imbalances(self, details, plug_account_id="Audit Sight Clearing", epsilon=0.01):
        """Append plug lines to fix overall/date/month imbalances before grouping.

        Strategy:
        - If per-date imbalances exist: add one plug per unbalanced date on that date
        - Else if per-month imbalances exist: add one plug per unbalanced month on the last date present in that month
        - Else if only overall imbalance exists: add one plug dated on the latest Posted Date in data
        Returns number of plugs added.
        """
        if self.journal_lines is None or len(self.journal_lines) == 0:
            return 0

        df = self.journal_lines
        next_row_index = int(df['_row_index'].max()) + 1 if '_row_index' in df.columns and len(df) > 0 else 0
        plugs_added = 0

        def append_plug_row(date_value, net):
            nonlocal next_row_index, plugs_added
            if abs(net) < epsilon:
                return
            plug_debit = 0.0
            plug_credit = 0.0
            if net > 0:
                plug_credit = abs(net)
            else:
                plug_debit = abs(net)
            plug = {
                'Posted Date': pd.to_datetime(date_value),
                'Account ID': plug_account_id,
                'Debit Amount': plug_debit,
                'Credit Amount': plug_credit,
                '_row_index': next_row_index,
            }
            for col in df.columns:
                if col not in plug:
                    plug[col] = None
            self.journal_lines = pd.concat([self.journal_lines, pd.DataFrame([plug])], ignore_index=True)
            next_row_index += 1
            plugs_added += 1
            self._plug_lines_added = True
            # Also track for output appending
            add_line = {col: None for col in self.journal_lines.columns if col != '_row_index'}
            add_line['Posted Date'] = pd.to_datetime(date_value)
            add_line['Account ID'] = plug_account_id
            add_line['Debit Amount'] = plug_debit
            add_line['Credit Amount'] = plug_credit
            self.additional_output_lines.append(add_line)

        # Prefer date-level fix when present
        unbalanced_dates = details.get('unbalanced_dates')
        if unbalanced_dates is not None and len(unbalanced_dates) > 0:
            for date_value, row in unbalanced_dates.iterrows():
                append_plug_row(date_value, float(row['net']))
            return plugs_added

        # Otherwise month-level fix
        unbalanced_months = details.get('unbalanced_months')
        if unbalanced_months is not None and len(unbalanced_months) > 0:
            df_tmp = self.journal_lines.copy()
            df_tmp['__month__'] = df_tmp['Posted Date'].dt.to_period('M')
            for month_period, row in unbalanced_months.iterrows():
                candidates = df_tmp[df_tmp['__month__'] == month_period]['Posted Date']
                if len(candidates) > 0:
                    date_value = pd.to_datetime(candidates.max())
                else:
                    date_value = pd.Period(month_period, freq='M').to_timestamp(how='end')
                append_plug_row(date_value, float(row['net']))
            return plugs_added

        # Otherwise overall only
        net = details.get('overall', {}).get('net', 0.0)
        if abs(net) >= epsilon:
            last_date = pd.to_datetime(self.journal_lines['Posted Date'].max())
            append_plug_row(last_date, float(net))
        return plugs_added

    def create_journal_entries(self, max_optional_fields=5):
        if self.journal_lines is None:
            print("❌ No data loaded")
            return False
        
        print("🔄 Creating journal entries...")
        
        optional_fields = self.get_optional_fields()
        print(f"📋 Found optional fields: {optional_fields}")
        
        field_combinations = self.generate_grouping_combinations(optional_fields, max_optional_fields)
        print(f"🔍 Testing {len(field_combinations)} grouping combinations...")
        
        assigned_lines = set()
        journal_entry_id = 1
        
        for fields in field_combinations:
            # Get unassigned lines and reset index to avoid groupby issues
            unassigned_df = self.journal_lines[~self.journal_lines['_row_index'].isin(assigned_lines)].copy().reset_index(drop=True)
            
            if len(unassigned_df) == 0:
                break
            
            # Dedupe and validate grouping fields
            valid_fields = [col for col in fields if col in unassigned_df.columns]
            valid_fields = list(dict.fromkeys(valid_fields))
            if not valid_fields:
                continue
            
            try:
                grouped = unassigned_df.groupby(valid_fields, dropna=False, sort=False)
            except Exception as e:
                print(f"   Error grouping by {valid_fields}: {e}")
                continue
            
            balanced_groups = 0
            
            for group_key, group_df in grouped:
                if self.check_balance(group_df):
                    je_id = self.generate_journal_id(valid_fields, group_key)
                    
                    group_df_clean = group_df.copy().reset_index(drop=True)
                    
                    self.grouped_entries[je_id] = {
                        'lines': group_df_clean,
                        'grouping_fields': valid_fields,
                        'group_key': group_key,
                        'total_debits': group_df_clean['Debit Amount'].sum(),
                        'total_credits': group_df_clean['Credit Amount'].sum()
                    }
                    
                    assigned_lines.update(group_df['_row_index'].tolist())
                    balanced_groups += 1
            
            if balanced_groups > 0:
                print(f"   ✅ Created {balanced_groups} entries with grouping: {valid_fields}")
        
        # Handle remaining lines
        remaining_lines = self.journal_lines[~self.journal_lines['_row_index'].isin(assigned_lines)].copy().reset_index(drop=True)
        
        if len(remaining_lines) > 0:
            print(f"📝 Processing {len(remaining_lines)} remaining lines...")
            
            for _, line in remaining_lines.iterrows():
                debit = line['Debit Amount']
                credit = line['Credit Amount']
                line_date = line['Posted Date']
                
                if debit == 0 and credit == 0:
                    # Try to attach zero-amount line to an existing JE on same date; otherwise leave unassigned
                    assigned_to_existing = False
                    for je_id, entry_data in self.grouped_entries.items():
                        if len(entry_data['lines']) > 0:
                            entry_date = entry_data['lines']['Posted Date'].iloc[0]
                            if entry_date.date() == line_date.date():
                                line_df = pd.DataFrame([line]).reset_index(drop=True)
                                existing_lines = entry_data['lines'].copy().reset_index(drop=True)
                                entry_data['lines'] = pd.concat([existing_lines, line_df], ignore_index=True)
                                entry_data['total_debits'] += line['Debit Amount']
                                entry_data['total_credits'] += line['Credit Amount']
                                assigned_lines.add(line['_row_index'])
                                assigned_to_existing = True
                                print(f"   ✅ Zero-amount line assigned to {je_id}")
                                break
                    if not assigned_to_existing:
                        # Keep unassigned; will be handled by plug balancing step
                        continue
                elif debit != 0 and credit != 0:
                    continue
                else:
                    # Do not create single-line entries; leave for plug balancing
                    continue
        
        self.unassigned_lines = self.journal_lines[~self.journal_lines['_row_index'].isin(assigned_lines)].copy()
        
        print(f"\\n📊 Summary:")
        print(f"   ✅ Journal entries created: {len(self.grouped_entries)}")
        print(f"   ✅ Lines assigned: {len(assigned_lines)}")
        print(f"   ⚠️  Invalid lines: {len(self.unassigned_lines)}")
        
        return True
    
    def generate_output(self, original_filename):
        if self.journal_lines is None:
            return None
        
        from openpyxl import load_workbook
        
        input_path = Path(original_filename)
        output_filename = f"{input_path.stem}_with_journal_ids{input_path.suffix}"
        
        # New template: write IDs into workbook and save
        if self.template_type == 'new' and self.workbook is not None:
            if not self.skip_id_creation:
                # Write Journal IDs - ALWAYS write to column A (column 1)
                jid_col_idx = 1
                # Set header in column A
                self.ws_jel.cell(row=self.ws_header_row, column=jid_col_idx, value='Journal ID')
                # Build complete header list for plug line mapping
                header_cells = self.ws_jel[self.ws_header_row]
                headers = [cell.value for cell in header_cells]
                
                # Find the last row with actual data (not just Journal IDs we might write)
                last_data_row = self.ws_data_start_row - 1
                for row in self.ws_jel.iter_rows(min_row=self.ws_data_start_row):
                    # Check if any cell in this row (excluding column A) has data
                    has_data = any(cell.value is not None for cell in row[1:])  # Skip column A
                    if has_data:
                        last_data_row = row[0].row
                    else:
                        break
                
                # Write Journal IDs to column A for all assigned lines (skip plug lines with _row_index < 0)
                for je_id, entry_data in self.grouped_entries.items():
                    row_indices = entry_data['lines']['_row_index'].tolist()
                    for idx in row_indices:
                        # Skip plug lines that were added during balancing (they have negative row index)
                        if int(idx) < 0:
                            continue
                        ws_row = self.ws_data_start_row + int(idx)
                        self.ws_jel.cell(row=ws_row, column=jid_col_idx, value=je_id)
                
                # Append plug lines to the bottom of the sheet - start right after last data row
                if self.additional_output_lines:
                    # Map column names to indices
                    col_to_idx = {}
                    for i, h in enumerate(headers):
                        if h:
                            col_to_idx[h] = i + 1
                    # Also need to map our internal column names to sheet columns
                    # Find Posted Date, Account ID, Debit Amount, Credit Amount columns
                    for i, h in enumerate(headers):
                        if h and 'posted' in str(h).lower() and 'date' in str(h).lower():
                            col_to_idx['Posted Date'] = i + 1
                        elif h and 'account' in str(h).lower() and 'id' in str(h).lower():
                            col_to_idx['Account ID'] = i + 1
                        elif h and 'debit' in str(h).lower():
                            col_to_idx['Debit Amount'] = i + 1
                        elif h and 'credit' in str(h).lower():
                            col_to_idx['Credit Amount'] = i + 1
                    
                    # Write each plug line row starting from last_data_row + 1
                    plug_row_start = last_data_row + 1
                    for idx, plug_line in enumerate(self.additional_output_lines):
                        new_row_idx = plug_row_start + idx
                        # Write Journal ID if present in plug_line dict
                        if 'Journal ID' in plug_line:
                            self.ws_jel.cell(row=new_row_idx, column=jid_col_idx, value=plug_line['Journal ID'])
                        # Write other fields
                        for key, val in plug_line.items():
                            if key == 'Journal ID':
                                continue
                            if key in col_to_idx:
                                self.ws_jel.cell(row=new_row_idx, column=col_to_idx[key], value=val)
                # Add clearing account to CTB if plug lines were added
                if self._plug_lines_added and self.ws_ctb is not None:
                    ctbr = self.ws_ctb.max_row + 1
                    self.ws_ctb.cell(row=ctbr, column=1, value='Audit Sight Clearing')
                    self.ws_ctb.cell(row=ctbr, column=2, value='Audit Sight Clearing')
                    self.ws_ctb.cell(row=ctbr, column=3, value=0)
                    self.ws_ctb.cell(row=ctbr, column=4, value=0)
                    self.ws_ctb.cell(row=ctbr, column=5, value='Assets')
                    self.ws_ctb.cell(row=ctbr, column=6, value='asset:current:other')
                    print("✅ Audit Sight Clearing account added to Comparative Trial Balances tab.")
            
            output_buffer = io.BytesIO()
            self.workbook.save(output_buffer)
            output_buffer.seek(0)
            self.print_summary_report()
            return output_buffer.getvalue(), output_filename
        
        # Old template: create new DataFrame output
        output_df = self.journal_lines.copy()
        output_df['Journal ID'] = ''
        
        for je_id, entry_data in self.grouped_entries.items():
            row_indices = entry_data['lines']['_row_index'].tolist()
            output_df.loc[output_df['_row_index'].isin(row_indices), 'Journal ID'] = je_id
        
        output_df = output_df.drop('_row_index', axis=1)
        
        # Append any additional output lines (plug lines)
        if self.additional_output_lines:
            cols_set = set(output_df.columns)
            add_rows = []
            for row in self.additional_output_lines:
                completed = {col: row.get(col, None) for col in output_df.columns}
                add_rows.append(completed)
            if add_rows:
                output_df = pd.concat([output_df, pd.DataFrame(add_rows)], ignore_index=True)
        
        cols = list(output_df.columns)
        cols.remove('Journal ID')
        posted_date_idx = cols.index('Posted Date')
        cols.insert(posted_date_idx, 'Journal ID')
        output_df = output_df[cols]
        
        output_buffer = io.BytesIO()
        output_df.to_excel(output_buffer, index=False)
        output_buffer.seek(0)
        
        self.print_summary_report()
        
        return output_buffer.getvalue(), output_filename
    
    def print_summary_report(self):
        print("\\n" + "="*50)
        print("📋 JOURNAL ENTRY SUMMARY")
        print("="*50)
        
        multi_line_entries = []
        single_line_entries = []
        
        for je_id, entry_data in sorted(self.grouped_entries.items()):
            if len(entry_data['lines']) > 1:
                multi_line_entries.append((je_id, entry_data))
            else:
                single_line_entries.append((je_id, entry_data))
        
        if multi_line_entries:
            print(f"\\n🔗 MULTI-LINE ENTRIES ({len(multi_line_entries)}):")
            for je_id, entry_data in multi_line_entries[:5]:
                lines = entry_data['lines']
                print(f"   {je_id}: {lines['Posted Date'].iloc[0].strftime('%Y-%m-%d')} | {len(lines)} lines | ${entry_data['total_debits']:,.2f}")
            
            if len(multi_line_entries) > 5:
                print(f"   ... and {len(multi_line_entries) - 5} more")
        
        if single_line_entries:
            print(f"\\n📄 SINGLE-LINE ENTRIES ({len(single_line_entries)}):")
            for je_id, entry_data in single_line_entries[:3]:
                lines = entry_data['lines']
                line = lines.iloc[0]
                print(f"   {je_id}: {line['Posted Date'].strftime('%Y-%m-%d')} | {line['Account ID']} | ${line['Debit Amount']:.2f}/${line['Credit Amount']:.2f}")
            
            if len(single_line_entries) > 3:
                print(f"   ... and {len(single_line_entries) - 3} more")

print("✅ Journal Entry Creator loaded successfully!")
print("\n🎯 Ready to process your Excel file!")
print("   ⬇️ Run the next cell to upload your file")


In [ ]:
# @title 📁 **STEP 2: Upload Your Excel File** { display-mode: "form" }
# @markdown Click ▶️ to upload your Excel file with journal line data.

print("📁 Upload Your Excel File")
print("=" * 30)
print("Your file should contain:")
print("   ✅ Posted Date column")
print("   ✅ Account ID column") 
print("   ✅ Debit Amount column")
print("   ✅ Credit Amount column")
print("   📋 Optional fields (Description, Reference, etc.)")
print()

# Upload file
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    file_data = uploaded[filename]
    print(f"\n✅ Successfully uploaded: {filename}")
    print(f"   📊 File size: {len(file_data):,} bytes")
    
    # Process immediately
    print("\n🚀 Processing your data...")
    print("=" * 40)
    
    creator = JournalEntryCreator()
    
    if creator.load_data_from_uploaded_file(file_data, filename):
        try:
            print("\n🧮 Validating balances (overall, by date, by month if needed)...")
            details = creator.validate_balances(return_details=True)
            if not details['ok']:
                print("❌ Balance validation failed:\n" + "\n\n".join(details['messages']))
                try:
                    resp = input("Add plug lines using 'Audit Sight Clearing' to fix these imbalances? [y/N]: ").strip().lower()
                except EOFError:
                    resp = 'n'
                if resp == 'y':
                    added = creator.add_plug_lines_for_imbalances(details)
                    print(f"✅ Added {added} plug line(s) to fix imbalances.")
                else:
                    raise Exception("User declined to auto-balance imbalances.")
        except Exception as e:
            print(f"❌ Balance validation failed:\n{e}")
        else:
            if creator.create_journal_entries():
                # If unassigned lines remain, prompt user to auto-balance with plug lines
                if len(creator.unassigned_lines) > 0:
                    print(f"\n⚠️ Found {len(creator.unassigned_lines)} unassigned lines after grouping.")
                    try:
                        resp = input("Add plug lines using 'Audit Sight Clearing' to balance each affected date? [y/N]: ").strip().lower()
                    except EOFError:
                        resp = 'n'
                    if resp == 'y':
                        balanced_dates = creator.balance_unassigned_with_plug()
                        print(f"✅ Added plug lines for {balanced_dates} posted date(s).")
                    else:
                        raise Exception("User declined to auto-balance unassigned lines.")
                
                output_data, output_filename = creator.generate_output(filename)
                
                print(f"\n🎉 SUCCESS! Your file is ready for download.")
                print(f"   📄 Output filename: {output_filename}")
                print("\n⬇️ Run the next cell to download your results!")
            else:
                print("❌ Failed to create journal entries")
    else:
        print("❌ Failed to load your Excel file")
else:
    print("❌ No file uploaded. Please run this cell again.")


In [ ]:
# @title 💾 **STEP 3: Download Your Results** { display-mode: "form" }
# @markdown Click ▶️ to download your processed Excel file with Journal IDs.

if 'output_data' in locals() and 'output_filename' in locals():
    print("💾 Downloading Your Results")
    print("=" * 30)
    
    # Save and download the file
    with open(output_filename, 'wb') as f:
        f.write(output_data)
    
    files.download(output_filename)
    
    print(f"✅ Download started: {output_filename}")
    print("\n📋 Your output file contains:")
    print("   ✅ All your original data")
    print("   ✅ New 'Journal ID' column")
    print("   ✅ Every line has a Journal ID")
    print("   ✅ Balanced journal entries")
    print("   ✅ Zero-amount lines properly assigned")
    
    print("\n🎯 Next Steps:")
    print("   📊 Import into your accounting system")
    print("   📈 Use Journal IDs for reporting")
    print("   🔍 Review the summary above")
    
    print("\n✨ Thank you for using Journal Entry ID Creator!")
    
else:
    print("❌ No processed data available.")
    print("   Please run the previous cell to upload and process your file first.")
